# Convolutional LSTM for coordinate prediction

### Imports

In [13]:
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from tqdm.auto import tqdm

# own
import common.action as action
import common.world as world
import common.plot as plot
import common.preprocess as preprocess
import common.nets as nets
import common.train as train
import common.tools as tools

### Load datasets

In [2]:
with open("datasets/oracle_data.pickle", "rb") as handle:
    oracle_data = pickle.load(handle)

with open("datasets/oracle_reversed_data.pickle", "rb") as handle:
    oracle_reversed_data = pickle.load(handle)

with open("datasets/oracle_random_data.pickle", "rb") as handle:
    oracle_random_data = pickle.load(handle)

### Preprocess data

In [3]:
length_trajectory = 10
batch_size = 128

# split and shuffle data
train_data, test_data = preprocess.split_data_for_trajectories(
    oracle_reversed_data, 0.8, length_trajectory
)
train_imgs, train_pos = preprocess.process_trajectory(train_data)
test_imgs, test_pos = preprocess.process_trajectory(test_data)

# stage data for the DataLoader
train_data = preprocess.ObtainDataset_notransform(train_imgs, train_pos)
test_data = preprocess.ObtainDataset_notransform(test_imgs, test_pos)

# DataLoader
dataset_loader_train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dataset_loader_test_data = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Initialize models

In [4]:
# initialize network
net_cnn = nets.CNN_coords()
net_lstm = nets.LSTM_coords(length_trajectory)

# initial values
h0 = torch.randn(2, 10, 100)
c0 = torch.randn(2, 10, 100)
x = torch.rand((batch_size, 10, 3, 32, 32))

# check network
features = net_cnn(x)
out0, out1, hidden, c = net_lstm(features, h0, c0)

# shape statistics
tools.shapes(x, features, hidden, out0)

# network summary
print("SUMMARY CNN \n", summary(net_cnn, (batch_size, 10, 3, 32, 32)), "\n")
print("SUMMARY LSTM \n", summary(net_lstm, ((batch_size, 10, 480), (2, 10, 100), (2, 10, 100))))

input cnn: torch.Size([128, 10, 3, 32, 32]) - Batch size, Channel out, Height out, Width out
output cnn: torch.Size([128, 10, 480])  - Batch size, sequence length, input size
input lstm: torch.Size([128, 10, 480])  - Batch size, sequence length, input size
hidden lstm: torch.Size([10, 100])
output lstm: torch.Size([128, 1]) 

SUMMARY CNN 
Layer (type:depth-idx)                   Output Shape              Param #
CNN_coords                               [128, 10, 480]            --
├─Conv2d: 1-1                            [1280, 10, 28, 28]        760
├─Conv2d: 1-2                            [1280, 20, 24, 24]        5,020
├─MaxPool2d: 1-3                         [1280, 20, 12, 12]        --
├─Conv2d: 1-4                            [1280, 30, 8, 8]          15,030
├─MaxPool2d: 1-5                         [1280, 30, 4, 4]          --
Total params: 20,810
Trainable params: 20,810
Non-trainable params: 0
Total mult-adds (G): 5.70
Input size (MB): 15.73
Forward/backward pass size (MB): 217.

In [5]:
print(len(out0))
print(len(out1))

128
128


In [11]:
inputs, label = next(iter(train_data))

#print('inputs', inputs)
print('len label', len(label))

print(label)


print(len(inputs))
inputs = torch.stack(inputs)
inputs = torch.swapaxes(inputs, 0, 1)
print(len(x))

net_cnn = nets.CNN_coords()

encoded = net_cnn(x)
print(encoded)

len label 10
[array([ 9.72609095,  0.        , 21.41735139]), array([ 9.57611305,  0.        , 21.41477643]), array([ 9.42613515,  0.        , 21.41220146]), array([ 9.27615726,  0.        , 21.40962649]), array([ 9.12617936,  0.        , 21.40705153]), array([ 8.97620146,  0.        , 21.40447656]), array([ 8.82622356,  0.        , 21.40190159]), array([ 8.67624567,  0.        , 21.39932662]), array([ 8.52626777,  0.        , 21.39675166]), array([ 8.37628987,  0.        , 21.39417669])]
10
128
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0788, 0.0358, 0.0690],
         [0.0000, 0.0000, 0.0000,  ..., 0.0818, 0.0445, 0.0410],
         [0.0000, 0.0000, 0.0000,  ..., 0.0249, 0.0453, 0.0950],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0472, 0.0569, 0.0500],
         [0.0000, 0.0000, 0.0000,  ..., 0.0534, 0.0738, 0.0513],
         [0.0000, 0.0000, 0.0000,  ..., 0.0628, 0.0556, 0.0293]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0623, 0.0413, 0.0469],
         [0.0000, 0.0000, 0.

### Train model

In [7]:
criterion = nn.MSELoss()
params = list(net_cnn.parameters()) + list(net_lstm.parameters())
optimizer = optim.Adam(params, lr=0.01)
episodes = 500

(
    train_loss,
    test_loss,
    train_dis,
    test_dis,
    train_dis_item,
    test_dis_item,
) = train.train_ConvLSTM(
    dataset_loader_train_data,
    dataset_loader_test_data,
    net_cnn,
    net_lstm,
    criterion,
    optimizer,
    episodes,
    length_trajectory,
)

Progress:   0%|          | 0/500 [00:00<?, ? Episode/s]

/Users/mariuskaestingschaefer/miniforge3/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128, 10])) that is different to the input size (torch.Size([128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (128) must match the size of tensor b (10) at non-singleton dimension 1

### Plot distance and loss over episodes

In [ ]:
plot.plot_euclidean_distance(train_dis, test_dis)
plot.plot_losses(train_loss[10:], test_loss[10:])

### Histogram of the distribution shift (for test and training distances)

In [ ]:
print("Training set \n")
plot.histo_distribution_shift(train_dis_item)
print("Validation set \n")
plot.histo_distribution_shift(test_dis_item)

### Histograms showing the training and validation distance distribution (for test and training distances)

In [ ]:
plot.histo_train_val(test_dis_item, train_dis_item)

### Save and load models